In [ ]:
import requests
zipurl = 'https://mtgjson.com/api/v5/AllPrintings.json.zip'
resp = requests.get(zipurl)

zname = "data/tmp/AllPrintings.json.zip"
zfile = open(zname, 'wb')
zfile.write(resp.content)
zfile.close()

from zipfile import ZipFile

# Create a ZipFile Object and load sample.zip in it
with ZipFile('data/tmp/AllPrintings.json.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('data/raw')

In [ ]:
import pandas as pd
import json

with open('data/raw/AllPrintings.json', encoding="utf8") as f:
    data = json.load(f)

In [ ]:
### PRINT OUT CSV JUST FOR KHM

results = []
set_info = []
editions = data['data'].keys()

set_name = data['data']['KHM']['name']
set_code = data['data']['KHM']['code']
cards_base_n = data['data']['KHM']['baseSetSize']
cards_n = data['data']['KHM']['totalSetSize']
calc_n = len(data['data']['KHM']['cards']) #add in check if the abs(calc_n - cards_n) <> 0
release_date = data['data']['KHM']['releaseDate']
    
set_info = {
    'set_name': set_name,
    'set_code': set_code,
    'release_date': release_date,
    'base set size': cards_base_n,
    'cards total N': cards_n,
    'cards total N (calc)': calc_n,
    'type': data['data']['KHM']['type'],
    'isOnlineOnly': data['data']['KHM']['isOnlineOnly']
}

    
for i in range(1,calc_n):
    card = data['data']['KHM']['cards'][i]['name']
    mtgjson_id = data['data']['KHM']['cards'][i]['identifiers']['mtgjsonV4Id']
    colors = data['data']['KHM']['cards'][i]['colors']

    card_info = {
        'set_name': set_name,
        'card_name': card,
        'mtgjson_id': mtgjson_id,
        'colors': colors,
        'hasNonFoil': data['data']['KHM']['cards'][i]['hasNonFoil'],
        'hasFoil': data['data']['KHM']['cards'][i]['hasFoil']
    }
    results.append(card_info)
    
results_df = pd.DataFrame.from_dict(results)

results_df.to_csv('data/out/test.csv')

In [ ]:
print(set_info)